#RFM Analysis in Python

https://thecleverprogrammer.com/2023/06/12/rfm-analysis-using-python/

RFM Analysis: Overview
RFM Analysis is a concept used by Data Science professionals, especially in the marketing domain for understanding and segmenting customers based on their buying behaviour.

Using RFM Analysis, a business can assess customers’:


*   recency (the date they made their last purchase)
*   frequency (how often they make purchases)
*   monetary value (the amount spent on purchases)


Recency, Frequency, and Monetary value of a customer are three key metrics that provide information about customer engagement, loyalty, and value to a business.

To perform RFM analysis using Python, we need a dataset that includes customer IDs, purchase dates, and transaction amounts. With this information, we can calculate RFM values for each customer and analyze their patterns and behaviours. I found an ideal dataset for this task

## Libraries

In [180]:
import pandas as pd

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

from datetime import datetime

## Import data

In [181]:
df = pd.read_csv('/content/drive/MyDrive/10_Learning/03_Python/Jupiter_root/RFM/data/rfm_data.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/10_Learning/03_Python/Jupiter_root/RFM/data/rfm_data.csv'

In [ ]:
df.dtypes

In [ ]:
# To manipulate PurchaseDate, we want to convert it into a date format
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])


## Recency

In [ ]:
# Calculate Recency
df['Recency'] = (datetime.now().date() - df['PurchaseDate'].dt.date).dt.days

## Frequency

In [ ]:
# Calculate Frequency
frequency_data = df.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID': 'Frequency'}, inplace=True)


In [ ]:
df = df.merge(frequency_data, on='CustomerID', how='left')

## Monetary

In [ ]:
monetary_data = df.groupby("CustomerID")["TransactionAmount"].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)


In [ ]:
df = df.merge(monetary_data, on="CustomerID", how="left")

## RFM - Scores

We assigned scores from 5 to 1 to calculate the recency score, where a higher score indicates a more recent purchase. It means that customers who have purchased more recently will receive higher recency scores.

We assigned scores from 1 to 5 to calculate the frequency score, where a higher score indicates a higher purchase frequency. Customers who made more frequent purchases will receive higher frequency scores.

To calculate the monetary score, we assigned scores from 1 to 5, where a higher score indicates a higher amount spent by the customer.



In [ ]:
# we use the function cut to split the data into 5 groups

df["Recency_score"] = pd.cut(x=df['Recency'], bins=5, labels=[5, 4, 3, 2, 1])
df["Frequency_score"] = pd.cut(x=df['Frequency'], bins=5, labels=[1, 2, 3, 4, 5])
df["Monetary_score"] = pd.cut(x=df['MonetaryValue'], bins=5, labels=[1, 2, 3, 4, 5])


In [ ]:
df["Recency_score"] = df["Recency_score"].astype(int)
df["Frequency_score"] = df["Frequency_score"].astype(int)
df["Monetary_score"] = df["Monetary_score"].astype(int)

## RFM Value Segmentation

We divided RFM scores into three segments, namely “Low-Value”, “Mid-Value”, and “High-Value”.

In [ ]:
df['RFM_Score'] = df['Recency_score'] + df['Frequency_score'] + df['Monetary_score']

In [ ]:
df["Value_Segment"] = pd.qcut(df["RFM_Score"],q=3 ,labels=['Low-Value', 'Mid-Value', 'High-Value'])

Visualisation

In [ ]:
segment_counts = df["Value_Segment"].value_counts().reset_index()
segment_counts.columns = ["Value_Segment","Frequency"]
segment_counts

In [ ]:
pastel_colors = px.colors.qualitative.Pastel

# Create the bar chart
fig_segment_dist = px.bar(segment_counts, x='Value_Segment', y='Frequency',
                          color='Value_Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')

# Update the layout
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                              yaxis_title='Frequency',
                              showlegend=False)

# Show the figure
fig_segment_dist.show()

## RFM_Customer_Segments

In [ ]:
#Mapping RFM_Customer_Segments
def Segments(value):
  if value >= 9:
    return 'Champions'
  elif value >= 6:
    return 'Potential Loyalists'
  elif value >= 5:
    return 'At Risk Customers'
  elif value >= 4:
    return 'Cannot Lose'
  elif value >= 3:
    return 'Lost'

#Apply function to RFM Score column:
df["RFM_Customer_Segments"]=df["RFM_Score"].apply(Segments)
#df["RFM_Customer_Segments2"]=df.apply(lambda x: Segments(x["RFM_Score"]), axis=1)

df.head()

## RFM Analysis

In [ ]:
segment_product_counts = df.groupby(['Value_Segment', 'RFM_Customer_Segments'])['CustomerID'].count().reset_index(name='Count')
segment_product_counts = segment_product_counts.sort_values(['Count'], ascending=[False])
segment_product_counts

In [ ]:
# Treemap of the count by Value Segment and RFM Customer Segments
fig_treemap_segment_product = px.treemap(segment_product_counts,
                                         path=['Value_Segment', 'RFM_Customer_Segments'],
                                         values='Count',
                                         color='Value_Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM_Customer_Segments by Value - Tree map')
fig_treemap_segment_product.show()

In [ ]:
#Bar plot
fig = px.bar(segment_product_counts, x='Value_Segment', y='Count', color='RFM_Customer_Segments', color_discrete_sequence=px.colors.qualitative.Pastel, title="RFM_Customer_Segments by Value - Bar Plot")
fig.show()

In [ ]:
def seg_dist(segment):
  segment_df = df[df['RFM_Customer_Segments'] == segment]

  fig = go.Figure()
  fig.add_trace(go.Box(y=segment_df['Recency_score'], name='Recency'))
  fig.add_trace(go.Box(y=segment_df['Frequency_score'], name='Frequency'))
  fig.add_trace(go.Box(y=segment_df['Monetary_score'], name='Monetary'))

  fig.update_layout(title='Distribution of RFM Values within Segment',
                    yaxis_title='RFM Value',
                    showlegend=True)

  return fig



In [ ]:
seg_dist('Champions')

In [ ]:
def seg_corr(segment):
  corr_df = df[df['RFM_Customer_Segments'] == segment]

  correlation_matrix = corr_df[['Recency_score', 'Frequency_score', 'Monetary_score']].corr()

  # Visualize the correlation matrix using a heatmap
  fig_heatmap = go.Figure(data=go.Heatmap(
                    z=correlation_matrix.values,
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    colorscale='RdBu',
                    colorbar=dict(title='Correlation')))

  fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within '+ segment)

  return fig_heatmap

In [ ]:
seg_corr('Champions')

In [ ]:
segment_counts = df['RFM_Customer_Segments'].value_counts()

In [ ]:
fig = go.Figure(go.Bar(x = segment_counts.index,y = segment_counts.values))

# Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()

In [ ]:
df

In [ ]:
Score = df[['RFM_Customer_Segments','Recency_score', 'Frequency_score', 'Monetary_score']].melt('RFM_Customer_Segments', value_name='Score')
Score.rename(columns={"variable":"Score_segment"}, inplace=True)

Score_mean = Score.groupby(['RFM_Customer_Segments',"Score_segment"])['Score'].mean().reset_index()


In [ ]:
fig = px.bar(Score_mean, x="RFM_Customer_Segments", y="Score",
             color="Score_segment", color_discrete_sequence=px.colors.qualitative.Pastel,
             barmode = 'group')
fig.show()